# IoU

To actually get the result we want we just need to call the metric_calc function,giving it as input the list of boxes (result of DETR) and the address of the heatmap

In [1]:
import cv2
import numpy as np

def metric_calc(box_list, img_address, tau):
    
    img = cv2.imread(img_address)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_norm = gray_norm / gray_norm.max()

    ### thresholded image
    _, thresh = cv2.threshold(gray_norm, tau, 1, cv2.THRESH_BINARY)
    
    tmp_box = boxinator(box_list.tolist(), img)
    tmp_img = thresh.astype(bool)
    max_IoU = 0
    max_IoU_ID = 0

    overlap = tmp_box * tmp_img
    union = tmp_box + tmp_img
    union[union > 1] = 1
    IoU = np.sum(overlap) / np.sum(union)
    return IoU

def boxinator(boxes, image):
    h, w, c = np.array(image).shape
    boxes = np.around(boxes).astype(int)

    box = np.array([np.array(image)[:, :, 0] * 0] * len(boxes), dtype=bool)  # 3D tensor

    for i in range(len(boxes)):
        # now we are looking at box i
        xmin, ymin, xmax, ymax = np.array(np.round_(boxes[i]), dtype=int)

        box[i][ymin:ymax, xmin:xmax] = 1

    bbl_mask = np.array(image)[:, :, 0] * 0

    for mask in box:
        bbl_mask += mask
        bbl_mask[bbl_mask > 1] = 1
    bbl_mask[bbl_mask > 1] = 1
    return np.squeeze(bbl_mask)

